# Algoritmo de Sumarização Baseado em Similaridade do Cosseno

### Preprocessamento

In [9]:
# !pip install spacy
# !python -m spacy download pt_core_news_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
import spacy
import re
import nltk
import string
import heapq
import numpy as np
from nltk.cluster.util import cosine_distance
import networkx as nx

In [6]:
pln = spacy.load('pt_core_news_sm')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
def preprocessamento(texto):
  stopwords = nltk.corpus.stopwords.words('portuguese')
  texto = texto.lower()
  texto = re.sub(r" +", ' ', texto)
  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)
  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

In [8]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas. 
                    Definem como o estudo de agente artificial com inteligência. 
                    Ciência e engenharia de produzir máquinas com inteligência. 
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [10]:
preprocessamento(texto_original)

'inteligência artificial ser inteligência similar humano máquina \n  definir comer estudar agente artificial inteligência \n  ciência engenhar produzir máquina inteligência \n  resolver problema possuir inteligência \n  relacionar comportamento inteligente \n  construção máquina parir raciocinar \n  aprender erro acerto \n  inteligência artificial ser raciocinar situação cotidiano'

### Função para calcular a similaridade entre as sentenças

In [12]:
def calcula_similaridade_sentencas(sentenca1, sentenca2):
  palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
  palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]
  todas_palavras = list(set(palavras1 + palavras2))
  vetor1 = [0] * len(todas_palavras)
  vetor2 = [0] * len(todas_palavras)
  for palavra in palavras1:
    vetor1[todas_palavras.index(palavra)] += 1
  for palavra in palavras2:
    vetor2[todas_palavras.index(palavra)] += 1
  return 1 - cosine_distance(vetor1, vetor2)

In [13]:
calcula_similaridade_sentencas(sentencas_formatadas[0], sentencas_formatadas[0])

1.0

### Função para gerar a matriz de similaridade

In [14]:
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
  for i in range(len(sentencas)):
    for j in range(len(sentencas)):
      if i == j:
        continue
      matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])
  return matriz_similaridade

In [16]:
calcula_matriz_similaridade(sentencas_formatadas)

array([[0.        , 0.40824829, 0.4472136 , 0.33333333, 0.        ,
        0.16666667, 0.        , 0.54433105],
       [0.40824829, 0.        , 0.18257419, 0.20412415, 0.        ,
        0.        , 0.        , 0.33333333],
       [0.4472136 , 0.18257419, 0.        , 0.2236068 , 0.        ,
        0.2236068 , 0.        , 0.18257419],
       [0.33333333, 0.20412415, 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.20412415],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.16666667, 0.        , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.20412415],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.54433105, 0.33333333, 0.18257419, 0.20412415, 0.        ,
        0.20412415, 0.        , 0.        ]])

### Função de Sumarização

In [17]:
def sumarizar(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])
  return sentencas_originais, melhores_sentencas, notas_ordenadas

### Teste da Função de Sumarização

In [19]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 3)

In [20]:
melhores_sentencas

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

### Sumarização de Artigos de Blog usando o Algoritmo de Baseado em Similaridade do Cosseno

In [21]:
from goose3 import Goose
g = Goose()
url = 'https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=inteligencia-artificial-descobre-leis-fisica-ocultas-dados&id=010150211230'
artigo = g.extract(url)

In [22]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [23]:
melhores_sentencas

['Sendo possível extrair dos dados as equações de movimento, então torna-se possível usar essas equações para criar modelos e simulações que sejam fiéis às leis da física.',
 'Hoje, o trabalho de fazer previsões de fenômenos físicos, como os do clima, é realizado por meio de modelos científicos, que são programados para criar simulações usando supercomputadores.',
 'Se equações de movimento puderem ser extraídas de tais casos, isso tornará possível fazer investigações e simulações mesmo desses fenômenos que até agora têm sido considerados impossíveis de se explicar usando a física.',
 'Yuhan Chen e seus colegas criaram uma ferramenta para fazer justamente isto, uma técnica para descobrir novas equações de movimento em dados observacionais para fenômenos aos quais as Leis de Newton podem ser aplicadas.',
 'Isso poderia ser usado para investigar a sustentabilidade do ecossistema, aplicando as leis físicas apropriadas (por exemplo, a lei de conservação de energia).']